In [1]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import joblib
import os

In [3]:
os.makedirs('model',exist_ok=True)
X,y=make_regression(n_samples=100,n_features=2,noise=0.1,random_state=42)
model=LinearRegression()
model.fit(X,y)
joblib.dump(model,'model/model.pkl')

['model/model.pkl']

In [4]:
loaded_model=joblib.load('model/model.pkl')
print("Reloaded model prediction:",loaded_model.predict([[0.1,0.2]]))

Reloaded model prediction: [23.58961171]


In [6]:
def predict(features,round_prediction=False):
    pred=loaded_model.predict([features])[0]
    if round_prediction:
        pred=round(pred,2)
    return pred
print("Function test:", predict([0.1, 0.2], round_prediction=True))

Function test: 23.59


In [7]:
import os
os.makedirs('src',exist_ok=True)
utils_code='''
def calculate_metrics(X,y):
    return {"mean_y":sum(y)/len(y),"sum_y":sum(y)}
'''
with open('src/utils.py','w') as f:
    f.write(utils_code)
print("Created /src/utils.py with calculate_metrics function.")
from src.utils import calculate_metrics
metrics=calculate_metrics([row[0] for row in X],y)
print("Metrics calculated from utility function:",metrics)

Created /src/utils.py with calculate_metrics function.
Metrics calculated from utility function: {'mean_y': np.float64(-7.614847620370264), 'sum_y': np.float64(-761.4847620370264)}


In [13]:
flask_code='''

from flask import Flask, request, jsonify, render_template_string
import joblib
import matplotlib.pyplot as plt
import io, base64

model = joblib.load('model/model.pkl')
app = Flask(__name__)

def predict(features):
    pred = model.predict([features])[0]
    return pred

@app.route('/predict', methods=['POST'])
def predict_default():
    data = request.get_json()
    features = data.get('features')
    return jsonify({'prediction': float(predict(features))})

@app.route('/predict/<float:input1>')
@app.route('/predict/<float:input1>/<float:input2>')
def predict_path(input1, input2=None):
    if input2 is None:
        features = [input1]
    else:
        features = [input1, input2]
    return jsonify({'prediction': float(predict(features))})

@app.route('/plot')
def plot_page():
    fig, ax = plt.subplots()
    ax.plot([0,1,2,3],[10,20,15,30])
    ax.set_title("Demo Plot")
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    html = f"<html><body><h1>Model Output Plot</h1><img src='data:image/png;base64,{img_base64}'></body></html>"
    return render_template_string(html)

if __name__ == '__main__':
    app.run(debug=False)
'''
with open('app.py','w') as f:
    f.write(flask_code)
print("Flask app saved to app.py")

Flask app saved to app.py


In [14]:
import os
# Windows command to launch Flask in separate terminal window
os.system("start cmd /k python app.py")
print("Flask server launched in external terminal window.")

Flask server launched in external terminal window.


In [15]:
streamlit_code = '''
import streamlit as st
import joblib

model = joblib.load('model/model.pkl')

st.title("Model Prediction Dashboard")

input_1 = st.number_input("Feature 1")
input_2 = st.number_input("Feature 2")

if st.button("Predict"):
    pred = model.predict([[input_1, input_2]])
    st.write(f"Prediction: {pred[0]}")
'''

with open('app_streamlit.py','w') as f:
    f.write(streamlit_code)

print("Streamlit app saved to app_streamlit.py")

# Launch Streamlit in external terminal (Windows example)
os.system("start cmd /k streamlit run app_streamlit.py")
print("Streamlit dashboard launched in external terminal window.")

Streamlit app saved to app_streamlit.py
Streamlit dashboard launched in external terminal window.


In [16]:
import requests

# POST request with JSON payload
response = requests.post("http://127.0.0.1:5000/predict", json={"features":[0.1, 0.2]})
print("POST response:", response.json())

# GET request with path parameters
response2 = requests.get("http://127.0.0.1:5000/predict/0.1/0.2")
print("GET response:", response2.json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000284A28EB310>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))